# Kuramoto: Fig. 6c


This script uses a trained model and the feedback control to create evaluation trajectories over many possible intial states sampled from a uniform distribution in $[0,\pi]$.

If you replace the distribution you many notice the robustess of NODEC to initial state settings.

Please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/kuramoto/gen_parameters.py```.

Please also make sure that a training proceedure has produced results in the corresponding paths used in plot and table scripts.
Running ```nodec_experiments/kuramoto/train.ipynb``` with default paths is expected to generate at the requiered location for the plots and table scripts in each folder.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.



## Imports

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os
os.sys.path.append('../../../')

import math
from copy import deepcopy

import torch
from torchdiffeq import odeint
import numpy as np
import pandas as pd

import networkx as nx


import plotly.express as px
from plotly import graph_objects as go

from nnc.controllers.neural_network.nnc_controllers import NNCDynamics
from nnc.controllers.baselines.oscillators.dynamics import AdditiveControlKuramotoDynamics
from nnc.controllers.baselines.oscillators.optimal_controllers import KuramotoFeedbackControl

from nnc.helpers.torch_utils.graphs import adjacency_tensor, maximum_matching_drivers, drivers_to_tensor
from nnc.helpers.torch_utils.oscillators import order_parameter_cos
from nnc.helpers.torch_utils.numerics import faster_adj_odeint
from nnc.helpers.plot_helper import ColorRegistry, base_layout
from nnc.helpers.torch_utils.evaluators import FixedInteractionEvaluator

from tqdm.cli import tqdm

## Load main parameters and data

In [ ]:
# Training parameters, such as device, float precision and whether a pre-trained model is used.
device = 'cpu'
dtype = torch.float

In [ ]:
# Loading Parameters for the graph
data_folder = '../../../../data/parameters/kuramoto/'
graph = 'erdos_renyi'
graph_folder = data_folder + graph + '/'

training_results_folder = '../../../../data/results/kuramoto/erdos_renyi/'

A = torch.load(graph_folder + 'adjacency.pt', map_location=device).float() # adjacency matrix
G = nx.from_numpy_matrix(A.numpy())
n_nodes = G.number_of_nodes()
mean_degree = np.mean(list(dict(G.degree()).values()))

A = A.to(device, dtype) # adjacency
L = A.sum(-1).diag() - A # laplacian

# to save results
results_folder = '../../../../results/kuramoto/' + graph + '/'
os.makedirs(results_folder, exist_ok=True)
os.makedirs(results_folder+'/sample_results/nodec', exist_ok=True)
os.makedirs(results_folder+'/sample_results/fc', exist_ok=True)

In [ ]:
# Load dynamics dependendent variables and states
coupling_constants = torch.load(data_folder + 'coupling_constants.pt', map_location=device).to(device, dtype)
frustration_constants = torch.load(data_folder + 'frustration_constants.pt', map_location=device).to(device, dtype)
natural_frequencies = torch.load(data_folder + 'nominal_angular_velocities.pt', map_location=device).to(device, dtype)
K = coupling_constants[2].item() # coupling constant, index 2 should be 0.4
frustration_constant = frustration_constants[0] # we use no frustration for this example
dynamics_params_folder = graph_folder + 'dynamics_parameters/coupling_' + '{:.1f}'.format(K) + '/'


x0 = 2*math.pi*torch.rand([100, n_nodes]).to(device=device, dtype=dtype) # you can try other distribution or
# initial states as well here.


# to avoid using extra memory we load the driver vector and use element-wise multiplication instead of the driver matrix.
gain_vector = torch.load(dynamics_params_folder + 'driver_vector.pt', map_location=device).to(device, dtype)
driver_nodes = torch.nonzero(gain_vector).cpu().numpy().flatten().tolist()
driver_percentage = len(driver_nodes)/len(gain_vector)
steady_state = torch.load(dynamics_params_folder + 'steady_state.pt', map_location=device).to(device, dtype)


In [ ]:
#  Controller parameters
# Feedback Control
feedback_control_constant = 10

# Neural Network training
n_hidden_units = 3

In [ ]:
print('Current experiment info:')
print('\t Loaded ' + graph + 'graph with: ' + str(n_nodes) + ' nodes and ' + str(G.number_of_edges()) + ' edges.' )
print('\t Coupling Constant: ' + str(K))
print('\t Frustration Constant: ' + str(frustration_constant.item()))
print('\t Natural Frequencies: mean: ' + str(natural_frequencies.mean().item()) + ' variance: ' + str(natural_frequencies.var().item()) )
print('\t Ratio of driver node vs total nodes: '  + str(len(driver_nodes)/n_nodes))
print('\t Feedback Control Constant: '  + str(feedback_control_constant))


In [ ]:
# Generating the dynamics:
dyn = AdditiveControlKuramotoDynamics(
    A, 
    K, 
    natural_frequencies,
    frustration_constant=frustration_constant
).to(device)

In [ ]:
# same architecture as in train
class EluFeedbackControl(torch.nn.Module):
    """
    Very simple Elu architecture for control of linear systems
    """
    def __init__(self, n_nodes, n_drivers, driver_matrix, n_hidden=3):
        super().__init__()
        self.linear = torch.nn.Linear(n_nodes,n_hidden)
        self.linear_h1 = torch.nn.Linear(n_hidden, n_hidden)
        self.linear_final = torch.nn.Linear(n_hidden, n_drivers)
        self.driver_matrix = driver_matrix

    def forward(self, t, x):
        """
        :param t: A scalar or a batch with scalars
        :param x: input_states for all nodes
        :return:
        """     
        u = self.linear(torch.sin(x))
        u = torch.nn.functional.elu(u)
        u = self.linear_h1(u)
        u = torch.nn.functional.elu(u)
        u = self.linear_final(u)
        # we multiply by the nn driver matrix to generate the control signal
        u = (self.driver_matrix@u.unsqueeze(-1)).squeeze(-1)
        return u

In [ ]:
# We convert the driver vector back to a matrix and convert the non-zero elements to 1, so that the neural network is agnostic of the exact gain values.
driver_matrix = drivers_to_tensor(A.shape[-1], driver_nodes).to(dtype=dtype, device=device)

In [ ]:
# load trained models parameters
neural_net = EluFeedbackControl(n_nodes, len(driver_nodes), 
                                driver_matrix,
                                n_hidden=n_hidden_units
                               ).to(dtype=dtype, device=device)
neural_net.load_state_dict(torch.load(training_results_folder +'trained_model.pt', map_location=device))

In [ ]:
# load the evaluation model
evaluation_steps = 5000
evalu = FixedInteractionEvaluator(
    exp_id='kuramoto_er',
    log_dir=None,
    n_interactions= evaluation_steps, # neither control works consistently for less than 2-3k steps
    loss_fn=lambda t,x: torch.tensor(order_parameter_cos(x[-1].cpu().detach())).mean(),
    ode_solver=None,
    ode_solver_kwargs={},
    preserve_intermediate_states=True,
    preserve_intermediate_controls=True,
    preserve_intermediate_times=True,
    preserve_intermediate_energies=True,
    preserve_intermediate_losses=True,
    preserve_params=False,
    preserve_init_loss = True
)

## Evaluation of both baselines on 100 samples

In [ ]:
n_samples = 100
nn_contorl_fun = lambda t,x: neural_net(t, x)
fc_contorl_fun = lambda t,x: (feedback_control_constant*gain_vector*torch.sin(-x)).detach()
total_time = 150

progress_bar = tqdm(range(n_samples))
for i in progress_bar:
    x0 = torch.rand([1, n_nodes], device='cuda', requires_grad = False)
    x0 = x0%(2*math.pi)
    with torch.no_grad():
        nnres = evalu.evaluate(dyn, 
                               nn_contorl_fun, 
                               x0.cpu(), 
                               total_time, 
                               -1
                              )
        torch.save(nnres, results_folder + 'nodec/sample_'+str(i)+'.pt')
        fcres =  evalu.evaluate(dyn, 
                               fc_contorl_fun, 
                               x0.cpu(), 
                               total_time, 
                               -1
                              )
        torch.save(fcres, results_folder + 'fc/sample_'+str(i)+'.pt')
